In [ ]:
#@title imports, helpers 
%tensorflow_version 2.x

import os
import io
import PIL.Image, PIL.ImageDraw, PIL.ImageFont
import base64
import zipfile
import json
import requests
import numpy as np
import matplotlib.pylab as pl
import glob
!pip install -q --progress-bar off einops
from einops import rearrange
!pip install -q --progress-bar off pykeops[colab] geomloss
from IPython.display import Image, HTML, clear_output
from tqdm import tqdm_notebook, tnrange
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision.models as models
from functools import partial

os.environ['FFMPEG_BINARY'] = 'ffmpeg'
import moviepy.editor as mvp
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter


def imread(url, max_size=None, mode=None):
  if url.startswith(('http:', 'https:')):
    # wikimedia requires a user agent
    headers = {
      "User-Agent": "Requests in Colab/0.0 (https://colab.research.google.com/; no-reply@google.com) requests/0.0"
    }
    r = requests.get(url, headers=headers)
    f = io.BytesIO(r.content)
  else:
    f = url
  img = PIL.Image.open(f)
  if max_size is not None:
    img.thumbnail((max_size, max_size), PIL.Image.ANTIALIAS)
  if mode is not None:
    img = img.convert(mode)
  img = np.float32(img)/255.0
  return img

def np2pil(a):
  if a.dtype in [np.float32, np.float64]:
    a = np.uint8(np.clip(a, 0, 1)*255)
  return PIL.Image.fromarray(a)

def imwrite(f, a, fmt=None):
  a = np.asarray(a)
  if isinstance(f, str):
    fmt = f.rsplit('.', 1)[-1].lower()
    if fmt == 'jpg':
      fmt = 'jpeg'
    f = open(f, 'wb')
  np2pil(a).save(f, fmt, quality=95)

def imencode(a, fmt='jpeg'):
  a = np.asarray(a)
  if len(a.shape) == 3 and a.shape[-1] == 4:
    fmt = 'png'
  f = io.BytesIO()
  imwrite(f, a, fmt)
  return f.getvalue()

def im2url(a, fmt='jpeg'):
  encoded = imencode(a, fmt)
  base64_byte_string = base64.b64encode(encoded).decode('ascii')
  return 'data:image/' + fmt.upper() + ';base64,' + base64_byte_string

def imshow(a, fmt='jpeg'):
  display(Image(data=imencode(a, fmt)))

def tile2d(a, w=None):
  a = np.asarray(a)
  if w is None:
    w = int(np.ceil(np.sqrt(len(a))))
  th, tw = a.shape[1:3]
  pad = (w-len(a))%w
  a = np.pad(a, [(0, pad)]+[(0, 0)]*(a.ndim-1), 'constant')
  h = len(a)//w
  a = a.reshape([h, w]+list(a.shape[1:]))
  a = np.rollaxis(a, 2, 1).reshape([th*h, tw*w]+list(a.shape[4:]))
  return a

def zoom(img, scale=4):
  img = np.repeat(img, scale, 0)
  img = np.repeat(img, scale, 1)
  return img

class VideoWriter:
  def __init__(self, filename='_autoplay.mp4', fps=30.0, **kw):
    self.writer = None
    self.params = dict(filename=filename, fps=fps, **kw)

  def add(self, img):
    img = np.asarray(img)
    if self.writer is None:
      h, w = img.shape[:2]
      self.writer = FFMPEG_VideoWriter(size=(w, h), **self.params)
    if img.dtype in [np.float32, np.float64]:
      img = np.uint8(img.clip(0, 1)*255)
    if len(img.shape) == 2:
      img = np.repeat(img[..., None], 3, -1)
    self.writer.write_frame(img)

  def close(self):
    if self.writer:
      self.writer.close()

  def __enter__(self):
    return self

  def __exit__(self, *kw):
    self.close()
    if self.params['filename'] == '_autoplay.mp4':
      self.show()

  def show(self, **kw):
      self.close()
      fn = self.params['filename']
      display(mvp.ipython_display(fn, **kw))

class LoopWriter(VideoWriter):
  def __init__(self, *a, fade_len=1.0, **kw):
    super().__init__(*a, **kw)
    self._intro = []
    self._outro = []
    self.fade_len = int(fade_len*self.params['fps'])

  def add(self, img):
    if len(self._intro) < self.fade_len:
      self._intro.append(img)
      return
    self._outro.append(img)
    if len(self._outro) > self.fade_len:
      super().add(self._outro.pop(0))
  
  def close(self):
    for t in np.linspace(0, 1, len(self._intro)):
      img = self._intro.pop(0)*t + self._outro.pop(0)*(1.0-t)
      super().add(img)
    super().close()


torch.set_default_tensor_type('torch.cuda.FloatTensor')

def to_nchw(img):
  img = torch.as_tensor(img)
  if len(img.shape) == 3:
    img = img[None,...]
  return img.permute(0, 3, 1, 2)

!nvidia-smi -L

In [ ]:
#@title VGG16-based feature extraction 
vgg16 = models.vgg16(pretrained=True).features

def calc_styles_vgg(imgs):
  style_layers = [1, 6, 11, 18, 25]  
  mean = torch.tensor([0.485, 0.456, 0.406])[:,None,None]
  std = torch.tensor([0.229, 0.224, 0.225])[:,None,None]
  x = (imgs-mean) / std
  grams = []
  for i, layer in enumerate(vgg16[:max(style_layers)+1]):
    x = layer(x)
    if i in style_layers:
      h, w = x.shape[-2:]
      y = x.clone()  # workaround for pytorch in-place modification bug(?)
      gram = torch.einsum('bchw, bdhw -> bcd', y, y) / (h*w)
      grams.append(gram)
  return grams

def create_vgg_loss(target_img):
  target_style = calc_styles_vgg(target_img)
  def loss_f(imgs):
    xs = calc_styles_vgg(imgs)
    loss = 0.0
    for x, y in zip(xs, target_style):
      loss = loss + (x-y).square().mean()
    return loss
  return loss_f

In [ ]:
# OT loss 
from geomloss import SamplesLoss
from torchvision.transforms.functional_tensor import gaussian_blur

def calc_styles_ot(img, ksize, leveln, show=False):
  levels = []
  for i in range(leveln):
    blured = gaussian_blur(img, [5, 5], [1, 1])
    unsharp = img + (img-blured)*2.0
    if show:
      imshow(unsharp[0].permute(1, 2, 0).cpu())
    win = F.unfold(unsharp, ksize).permute(0, 2, 1).contiguous()
    levels.append(win)
    if i<leveln-1:
      img = blured[:,:,::2,::2]
  return levels
 
opt_f = SamplesLoss()

def subsample(x, n=2048):
  if x.shape[1]>n:
    idx = torch.multinomial(torch.ones(x.shape[1]), n)
    x = x[:,idx]
  return x

def create_ot_loss(target_img, ksize=5, leveln=5, show=False):
  target_style = calc_styles_ot(target_img, ksize, leveln, show)
  def loss_f(imgs):
    xs = calc_styles_ot(imgs, ksize, leveln)
    loss = 0.0
    for x, y in zip(xs, target_style):
      x, y = subsample(x, 1024), subsample(y, 1024*2)
      y = y.repeat(x.shape[0], 1, 1)
      loss += opt_f(x, y).mean()
    return loss
  return loss_f

In [ ]:
# μNCA 
side = torch.tensor([[0.0, 0.0,0.0], [2.0,-2.0,0.0], [0.0, 0.0,0.0]]) 
sobel_x = torch.tensor([[-1.0,0.0,1.0],[-2.0,0.0,2.0],[-1.0,0.0,1.0]]) 
lap = torch.tensor([[1.0,2.0,1.0],[2.0,-12,2.0],[1.0,2.0,1.0]])

# filters = [lap]*2+[sobel_x]*1+[sobel_x.T]*1  # 68 params
# filters = [lap]*2+[sobel_x]*2+[sobel_x.T]*2 # 150 params
# filters = [lap]*4+[sobel_x]*2+[sobel_x.T]*2  # 264 params
filters = [lap]*4+[sobel_x]*4+[sobel_x.T]*4  # 588 params

filters = torch.stack(filters)[:,None]
CHN = len(filters)

class CA(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.w = torch.nn.Parameter(torch.randn(CHN, 4*CHN+1, 1, 1)*1e-3)

  def to_rgb(self, x):
    return x[...,:3,:,:]+0.5

  def forward(self, x, update_rate=1.0):
    y = torch.nn.functional.pad(x, [1, 1, 1, 1], 'circular')
    y = torch.nn.functional.conv2d(y, filters, groups=y.shape[1])
    y = torch.cat([x, y], 1)
    y = y = torch.cat([y, y.abs()], 1)
    #y = torch.cat([y.relu(), -(-y).relu()], 1)
    w, b = self.w[:,:-1], self.w[:,-1,0,0]
    y = torch.nn.functional.conv2d(y, w, b)
    if update_rate<1.0:
      y *= (torch.rand(*y.shape)+update_rate).floor()
    return x+y, y
    
def seed_f(n, sz=128):
  return torch.rand(n, CHN, sz, sz)-0.5

ca = CA()
print('param count:', sum(p.numel() for p in ca.parameters()))

In [ ]:
# target
url = 'https://www.robots.ox.ac.uk/~vgg/data/dtd/thumbs/bubbly/bubbly_0101.jpg'

with torch.no_grad():
  style_img = imread(url, max_size=128)
  texture_loss_f = create_ot_loss(to_nchw(style_img))
imshow(style_img)

In [ ]:
#@title setup training
ca = CA() 
opt = torch.optim.Adam(ca.parameters(), 1e-3)
lr_sched = torch.optim.lr_scheduler.MultiStepLR(opt, [1000, 4000], 0.3)
loss_log = []
with torch.no_grad():
  pool = seed_f(256)
!rm *.pt

In [ ]:
torch.backends.cudnn.benchmark = True

In [ ]:
#@title training loop {vertical-output: true}

def loss_f(x):
  loss = texture_loss_f(ca.to_rgb(x))
  reg = (x-x.clamp(-1.0, 1.0)).square().sum()
  total_loss = loss + reg
  return total_loss, loss

best_long_loss = np.inf
best_long_step = 0

for i in range(10000):
  batch_idx = np.random.choice(len(pool), 4, replace=False)
  x = pool[batch_idx]
  if i%8 == 0:   
    x[:1] = seed_f(1)
  step_n = np.random.randint(32, 128)
  torch.autograd.set_grad_enabled(True)
  for _ in range(step_n):
    x, dx = ca(x)
    x.register_hook(partial(F.normalize, dim=[1,2,3]))
  total_loss, loss = loss_f(x)
  total_loss.backward()
  with torch.no_grad():
    for p in ca.parameters():
      p.grad /= (p.grad.norm()+1e-8)   # normalize gradients 
    opt.step()
    opt.zero_grad()
    lr_sched.step()
    pool[batch_idx] = x                # update pool
    
    loss_log.append(loss.item())
    if i%64==0:
      clear_output(True)

      min_loss = np.min(loss_log)
      pl.plot(loss_log, '.', alpha=0.1)
      if min_loss > 0.0:
        pl.yscale('log')
      pl.ylim(min_loss, loss_log[len(loss_log)//5])
      pl.show()
      imgs = ca.to_rgb(x).permute([0, 2, 3, 1]).cpu()
      imshow(np.hstack(imgs))
      pl.show()
      p = ca.w.cpu()[:,:,0,0]
      vis = torch.stack([(-p).relu(), p.relu(), 0.0*p], -1)
      vis *= 1.0/vis.max()
      imshow(zoom(vis))
      pl.show()

      # show long-term behaviour
      x = seed_f(1)
      for _ in range(3000):
        x, _ = ca(x)
      img = ca.to_rgb(x)[0].permute(1, 2, 0).cpu()
      imshow(img)
      _, long_loss = loss_f(x.clip(-1, 1))
      if long_loss<best_long_loss and len(loss_log)-best_long_step > 100:
        torch.save(ca, 'ca_%05d.pt'%len(loss_log))
        best_long_loss = long_loss
        best_long_step = len(loss_log)

    if i%10 == 0:
      print('\rstep_n:', len(loss_log),
        ' loss:', loss.item(), 
        ' lr:', lr_sched.get_lr()[0], end='')

In [ ]:
# μNCA pattern after 1000 steps 
with torch.no_grad():
  x = seed_f(1, 128)
  ca1 = torch.load('ca_%05d.pt'%best_long_step)
  for k in tnrange(1000, leave=False):
    x, _ = ca1(x)
  img = ca1.to_rgb(x)[0].permute(1, 2, 0).cpu()
  imshow(zoom(img, 2))

In [ ]:
#@title μNCA dynamics evolution video {vertical-output: true}
with torch.no_grad():
  ca1 = torch.load('ca_%05d.pt'%best_long_step)
with LoopWriter('final_ca.mp4', fade_len=0.5) as vid, torch.no_grad():
  x = seed_f(1, 256)
  #x[:] = 0.0
  #x[:,:,100, 100] = 1.0
  for k in tnrange(400, leave=False):
    img = ca.to_rgb(x)[0].permute(1, 2, 0).cpu()
    vid.add(zoom(img, 2))
    step_n = int(min(2**(k/30), 8))
    for i in range(step_n):
      x, _ = ca1(x)
vid.show()

In [ ]:
# export 68-param to ShaderToy
if len(filters) == 4:
  S = 1000.0
  w = (ca1.w[:,:,0,0]*S).round()
  mat4 = 'mat4(%s)'%('%d,'*16)[:-1]
  ws = ["%.0e"%(1/S), 'vec4(%d,%d,%d,%d)'%tuple(w[:,-1])]+[mat4%tuple(P.ravel()) for P in w.T[:-1].reshape(4, 4, 4)]
  code = '''vec4 rule(vec4 s, vec4 p) {
    return %s*(%s+
      %s*s+
      %s*p+
      %s*abs(s)+
      %s*abs(p));
  }'''%tuple(ws)
  print(code)

In [ ]:
# export 588-param to ShaderToy
if len(filters) == 12:
  w_scale = 500
  with torch.no_grad():
    w, b = ca1.w[:,:-1,0,0], ca1.w[:,-1,0,0]
    weight = (w.T*w_scale).round().cpu()
    bias = (b*w_scale).round().cpu()
  
  code = ['''
  vec4 update(float band, vec4 y[6]) {
    #define M mat4
    #define F(i,_a,_b) {M a=_a,b=_b; vec4 yi=y[i]; dx+=G(0)+G(1)+G(2)+G(3);}
    #define G(i) yi[i]*((yi[i]>0.0)?a[i]:b[i])
    vec4 dx;''']
  mat4_fmt = 'M(%s)'%','.join(['%d']*16)
  for i, band in enumerate(np.split(weight, 3, 1)):
    s = ' dx = vec4(%d,%d,%d,%d);'%tuple(bias[i*4:i*4+4])
    code.append('  '+('if (band == 0.) {', '} else if (band == 1.) {', '} else {')[i] + s)
    for j in range(CHN*2//4):
      a = mat4_fmt % tuple(band[j*4:][:4].ravel())
      b = mat4_fmt % tuple(band[j*4+CHN*2:][:4].ravel())
      code.append(f'    F({j}, {a}, {b});')
  code.append('''  }
    #undef M
    #undef F
    #undef G
    return dx/%.1f;
  }'''%w_scale)
  code = '\n'.join(code) 
  print(len(code))
  print(code)